# Benchmarking with Our Test Split

This notebook details how you can benchmark with our test split using any function.

## Installs and Imports

In [14]:
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
# !sudo yum install git-lfs -y
# !git lfs install

In [6]:
%%capture
%pip install torch==1.9.0+cpu torchvision==0.10.0+cpu torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# %pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
%pip install datasets==1.11.0
%pip install transformers==4.9.1
%pip install jiwer
%pip install ipywidgets
%pip install pythainlp==2.3.1
%pip install pydub
%pip install SpeechRecognition
%pip install azure-cognitiveservices-speech
%pip install deepcut

In [71]:
from functools import partial
import pandas as pd
import numpy as np
from datasets import (
    load_dataset, 
    load_from_disk,
    load_metric,)
from datasets.filesystems import S3FileSystem
from transformers import (
    Wav2Vec2CTCTokenizer, 
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
)
import torch
import torchaudio
import re
import json
from tqdm.auto import tqdm

## Download Dataset

We download the whole Common Voice 7.0 even though we will use only our test split for benchmarking. We also copy `train_cleaned.tsv`, `dev_cleaned.tsv`, and `test_cleaned.tsv` for our cleaned and resampled splits.

In [2]:
# !wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-7.0-2021-07-21/cv-corpus-7.0-2021-07-21-th.tar.gz
# !tar -xvf cv-corpus-7.0-2021-07-21-th.tar.gz --no-same-owner
# !mv cv-corpus-7.0-2021-07-21-th ../data
# !cp train_cleaned.tsv dev_cleaned.tsv test_cleaned.tsv ../data/cv-corpus-7.0-2021-07-21-th/th

## Load Test Set

In [72]:
test_dataset = load_dataset("../scripts/th_common_voice_70.py", "th", split="test")
test_dataset

Reusing dataset common_voice (/home/ec2-user/.cache/huggingface/datasets/common_voice/th/7.0.0/14bf435a174687b310ed94f56abf0198f6cc7efb5a5d945c22c83113eab67701)


Dataset({
    features: ['path', 'sentence'],
    num_rows: 2502
})

## Load Metrics

We use WER with words tokenized by PyThaiNLP 2.3.1 and deepcut, and CER.

In [73]:
from pythainlp.tokenize import word_tokenize
import deepcut

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

## Preprocess

In [74]:
def speech_file_to_array_fn(batch, 
                            text_col="sentence", 
                            fname_col="path",
                            resampling_to=16000):
    speech_array, sampling_rate = torchaudio.load(batch[fname_col])
    resampler=torchaudio.transforms.Resample(sampling_rate, resampling_to)
    batch["speech"] = resampler(speech_array)[0].numpy()
    batch["sampling_rate"] = resampling_to
    batch["target_text"] = batch[text_col]
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

  0%|          | 0/2502 [00:00<?, ?ex/s]

## Evaluate

### Convert to `.wav` for Some APIs

We need to convert the `mp3` files to `wav`. Install `ffmpeg and ffprobe`:

In [ ]:
# sudo su -
# mkdir -v -p /usr/local/bin/ffmpeg
# cd /usr/local/bin/ffmpeg
# wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-i686-static.tar.xz
# tar -v -xf ffmpeg-release-i686-static.tar.xz --strip-components=1
# rm -v -f ffmpeg-release-i686-static.tar.xz
# ln -snf /usr/local/bin/ffmpeg/ffmpeg /usr/bin/ffmpeg
# ln -snf /usr/local/bin/ffmpeg/ffprobe /usr/bin/ffpropbe
# exit

In [8]:
from pydub import AudioSegment

def mp3_to_wav(example):
    example['path_wav'] = (example['path'][:-3] + 'wav').replace('clips','clips_wav')
    sound = AudioSegment.from_mp3(example['path'])
    sound.export(example['path_wav'], format="wav")
    return example

In [9]:
test_dataset = test_dataset.map(mp3_to_wav)
test_dataset

  0%|          | 0/2502 [00:00<?, ?ex/s]

Dataset({
    features: ['path', 'sentence', 'speech', 'sampling_rate', 'target_text', 'path_wav'],
    num_rows: 2502
})

### [Google Web Speech API](https://w3c.github.io/speech-api/speechapi.html)

In [118]:
import speech_recognition as sr

r = sr.Recognizer()

def evaluate_example(fname):
    with sr.AudioFile(fname) as source:
        audio_text = r.listen(source)
    text = r.recognize_google(audio_text, language = "th-TH")
    return text
        
evaluate_example(test_dataset[0]['path_wav'])

'และเขาก็สัมผัสดีบุก'

In [ ]:
ds = []
for i in tqdm(test_dataset):
    try:
        pred = evalute_example(i['path_wav']).replace(' ','')
    except:
        pred = ''
    d = {'sentence': i['sentence'].replace(' ',''), 
         'pred_sentence': pred}
    d['sentence_tok'] = ' '.join(word_tokenize(d['sentence']))
    d['pred_sentence_tok'] = ' '.join(word_tokenize(d['pred_sentence']))
    ds.append(d)

  0%|          | 0/2502 [00:00<?, ?it/s]

In [18]:
# result_df = pd.DataFrame(ds)
# result_df['sentence_deepcut'] = result_df['sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df['pred_sentence_deepcut'] = result_df['pred_sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df.to_csv('artifacts/result_google.csv',index=False)

result_df = pd.read_csv('artifacts/result_google.csv').fillna('')
result_df

,sentence,pred_sentence,sentence_tok,pred_sentence_tok,sentence_deepcut,pred_sentence_deepcut
0,และเขาก็สัมผัสดีบุก,และเขาก็สัมผัสดีบุก,และ เขา ก็ สัมผัส ดีบุก,และ เขา ก็ สัมผัส ดีบุก,และ เขา ก็ สัมผัส ดี บุก,และ เขา ก็ สัมผัส ดี บุก
1,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณสามารถรับทราบข้อความนี้ถูกอ่านแล้ว,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...,คุณ สามารถ รับทราบ ข้อความ นี้ ถูก อ่าน แล้ว,คุณ สามารถ รับ ทราบ เมื่อ ข้อความ นี้ ถูก อ่าน...,คุณ สามารถ รับ ทราบ ข้อความ นี้ ถูก อ่าน แล้ว
2,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืน นี้ ฉัน ต้อง ทำ ให้ ได้ เธอ พูด กับ ตัว เอ...,คืน นี้ ฉัน ต้อง ทำ ให้ ได้ เธอ พูด กับ ตัว เอ...
3,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงฟลูออเรสเ...,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงตัวartists,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่น นี้ จะ ทำ ให้ แผ่น ถ่าย ภาพ เกิด แ...,การ ทำ เช่น นี้ จะ ทำ ให้ แผ่น ถ่าย ภาพ เกิด แ...
4,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ ปกครอง ของ ฉัน ไม่ สนใจ ความ เห็น นี้ อย่า...,ผู้ ปกครอง ของ ฉัน ไม่ สนใจ ความ เห็น นี้ อย่า...
...,...,...,...,...,...,...
2497,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใครสนใจจะเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ จะ เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ จะ เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...
2498,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้ง ครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้ง ครรภ์ เมื่อ อายุ เกิน
2499,ก่อนสเต็ปนั้น,ก่อนสเต็ปนั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น
2500,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำ พูด จะ นำ ไป สู...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำ พูด จะ นำ ไป สู...


In [19]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.1371123407540857

In [20]:
#wer deepcut
wer_metric.compute(predictions=result_df.pred_sentence_deepcut,references=result_df.sentence_deepcut)

0.10860058309037901

In [21]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.07357340720221607

### [Microsoft Bing Speech API](https://azure.microsoft.com/en-us/services/cognitive-services/speech/)

In [ ]:
import azure.cognitiveservices.speech as speechsdk

API_KEY = 'YOUR_API_KEY'
REGION = 'YOUR_REGION'
speech_config = speechsdk.SpeechConfig(subscription=API_KEY, 
                                       region=REGION)
speech_config.speech_recognition_language = "th-TH"

def evaluate_example(fname):
    audio_input = speechsdk.AudioConfig(filename=fname)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
    result = speech_recognizer.recognize_once_async().get()
    return result.text

evaluate_example(test_dataset[0]['path_wav'])

In [ ]:
ds = []
for i in tqdm(test_dataset):
    try:
        pred = evaluate_example(i['path_wav']).replace(' ','')
    except:
        pred = ''
    d = {'sentence': i['sentence'].replace(' ',''), 
         'pred_sentence': pred}
    d['sentence_tok'] = ' '.join(word_tokenize(d['sentence']))
    d['pred_sentence_tok'] = ' '.join(word_tokenize(d['pred_sentence']))
    ds.append(d)

In [23]:
# result_df = pd.DataFrame(ds)
# result_df['sentence_deepcut'] = result_df['sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df['pred_sentence_deepcut'] = result_df['pred_sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df.to_csv('artifacts/result_bing.csv',index=False)

result_df = pd.read_csv('artifacts/result_bing.csv').fillna('')
result_df

,sentence,pred_sentence,sentence_tok,pred_sentence_tok,sentence_deepcut,pred_sentence_deepcut
0,และเขาก็สัมผัสดีบุก,และเขาก็สัมผัสดีบุ๊ค,และ เขา ก็ สัมผัส ดีบุก,และ เขา ก็ สัมผัส ดี บุ๊ค,และ เขา ก็ สัมผัส ดี บุก,และ เขา ก็ สัมผัส ดี บุ๊ค
1,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...,คุณ สามารถ รับ ทราบ เมื่อ ข้อความ นี้ ถูก อ่าน...,คุณ สามารถ รับ ทราบ เมื่อ ข้อความ นี้ ถูก อ่าน...
2,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืน นี้ ฉัน ต้อง ทำ ให้ ได้ เธอ พูด กับ ตัว เอ...,คืน นี้ ฉัน ต้อง ทำ ให้ ได้ เธอ พูด กับ ตัว เอ...
3,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงฟลูออเรสเ...,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงตัวริมเส้น,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่น นี้ จะ ทำ ให้ แผ่น ถ่าย ภาพ เกิด แ...,การ ทำ เช่น นี้ จะ ทำ ให้ แผ่น ถ่าย ภาพ เกิด แ...
4,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ ปกครอง ของ ฉัน ไม่ สนใจ ความ เห็น นี้ อย่า...,ผู้ ปกครอง ของ ฉัน ไม่ สนใจ ความ เห็น นี้ อย่า...
...,...,...,...,...,...,...
2497,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...
2498,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้ง ครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้ง ครรภ์ เมื่อ อายุ เกิน
2499,ก่อนสเต็ปนั้น,ก่อนสเต็ปนั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น
2500,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำ พูด จะ นำ ไป สู...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำ พูด จะ นำ ไป สู...


In [24]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.12578818684853943

In [25]:
#wer deepcut
wer_metric.compute(predictions=result_df.pred_sentence_deepcut,references=result_df.sentence_deepcut)

0.09620991253644315

In [13]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.050166204986149585

## [AWS Transcribe](https://aws.amazon.com/transcribe/)

In [ ]:
import time
import boto3

s3 = boto3.resource("s3")
s3_bucket = s3.Bucket("cv-test-unique")
prefix = "th/clips_wav"
clips = [f.key.split('/')[-1] for f in s3_bucket.objects.filter(Prefix=prefix).all()]
clips = [i for i in clips if i[-3:]=='wav']
len(clips)

In [ ]:
ds = []

for i in tqdm(range(len(clips)//bs+1)):
    sub_clips = clips[i*bs:(i+1)*bs]
    for job_name in sub_clips:
        job_uri = f's3://cv-test-unique/{prefix}/{job_name}'
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat='wav',
            LanguageCode='th-TH'
        )

    while True:
        dones = []
        for job_name in sub_clips:
            status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
            dones.append(status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED'])
        if np.mean(dones)==1:
            print('Batch done')
            break
        print(f"{sum(dones)} out of {len(dones)} done for batch {i}")
        time.sleep(5)


    for job_name in sub_clips:
        status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        try:
            data = pd.read_json(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
            pred_text = data['results'][1][0]['transcript']
        except:
            pred_text = ''
        ds.append((job_name, ''.join(pred_text.split())))

    df = pd.DataFrame(ds)
    df.columns = ['fname','pred_sentence']
    df.to_csv(f'result_aws_{i}.csv',index=False)

In [35]:
# ground_truth = pd.DataFrame(test_dataset)
# ground_truth['fname'] = ground_truth.path.map(lambda x: x.split('/')[-1])
# df['fname'] = df['fname'].map(lambda x: x.replace('.wav','.mp3'))
# result_df = ground_truth.merge(df,on='fname')
# result_df = result_df.drop(['fname','path'],1)
# result_df['pred_sentence'] = result_df['pred_sentence'].map(lambda x: x.replace('.','').replace(' ',''))
# result_df['sentence'] = result_df['sentence'].map(lambda x: x.replace(' ',''))
# result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
# result_df['pred_sentence_tok'] = result_df.pred_sentence.map(lambda x: ' '.join(word_tokenize(x)))
# result_df['sentence_deepcut'] = result_df['sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df['pred_sentence_deepcut'] = result_df['pred_sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df.to_csv('artifacts/result_aws.csv',index=False)

In [30]:
result_df = pd.read_csv('artifacts/result_aws.csv').fillna('')

In [37]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.21863338051730793

In [34]:
#wer deepcut
wer_metric.compute(predictions=result_df.pred_sentence_deepcut,references=result_df.sentence_deepcut)

0.14487553263063468

In [13]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.07077562326869806

### [NECTEC AI for Thai Partii API](https://aiforthai.in.th/aiplatform/#/speechtotext)

We benchmark with the Partii API using instructions listed in [AI for Thai webiste](https://aiforthai.in.th/aiplatform/#/speechtotext).

In [119]:
import requests
 
url = "https://api.aiforthai.in.th/partii-webapi"
 
fname = test_dataset[0]['path_wav']
files = {'wavfile': (fname, open(fname, 'rb'), 'audio/wav')}

API_KEY = 'YOUR_API_KEY'
 
headers = {
    'Apikey': API_KEY,
    'Cache-Control': "no-cache",
    'Connection': "keep-alive",
    }
 
param = {"outputlevel":"--uttlevel","outputformat":"--txt"}
 
response = requests.request("POST", url, headers=headers, files=files, data=param)
 
print("Result = " + response.text)

Result = {"status":"False","message":"No asr result","inputfilename":"","outputfilename":""}


In [38]:
# #prediction
# df = pd.read_csv('artifacts/result_partii.tsv',sep='\t') #result called by vistec-ai/nectec-POSTer by @tann9949
# df.columns = ['fname','text']
# df['fname'] = df['fname'].map(lambda x: x+'.mp3')
# df['pred_sentence'] = df.text.map(lambda x: ''.join(x.replace(' ','').split()))
# df['pred_sentence_tok'] = df.pred_sentence.map(lambda x: ' '.join(word_tokenize(x)))
# df = df[['fname','pred_sentence','pred_sentence_tok']]

# #ground truth
# ground_truth = pd.DataFrame({'path':test_dataset['path'], 'sentence':test_dataset['sentence']})
# ground_truth['fname'] = ground_truth.path.map(lambda x: x.split('/')[-1])

# #merge
# result_df = ground_truth.merge(df,on='fname')
# result_df = result_df.drop(['fname','path'],1)
# result_df['sentence'] = result_df['sentence'].map(lambda x: x.replace(' ',''))
# result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
result_df['sentence_deepcut'] = result_df['sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
result_df['pred_sentence_deepcut'] = result_df['pred_sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
result_df.to_csv('artifacts/result_partii.csv',index=False)

In [37]:
result_df = pd.read_csv('artifacts/result_partii.csv').fillna('')

In [39]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.2010588680196647

In [40]:
#wer deepcut
wer_metric.compute(predictions=result_df.pred_sentence_deepcut,references=result_df.sentence_deepcut)

0.15515631009285205

In [26]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.0955102705784922

### `wav2vec2` Models from HuggingFace

In [75]:
processor = Wav2Vec2Processor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
model = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

def evaluate(batch):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to(device),).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_sentence"] = processor.batch_decode(pred_ids)
    return batch

Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [78]:
%%time
#takes quite a long time on CPU
result = test_dataset.map(evaluate, batched=True, batch_size=8)

  0%|          | 0/313 [00:00<?, ?ba/s]

CPU times: user 2h 41min 59s, sys: 18min 57s, total: 3h 57s
Wall time: 45min 31s


In [25]:
result_df = pd.DataFrame({'sentence':result['sentence'], 'pred_sentence_tok': result['pred_sentence']})
result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
result_df['pred_sentence'] = result_df.pred_sentence_tok.map(lambda x: ''.join(x.split()))
result_df['sentence_deepcut'] = result_df['sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
result_df['pred_sentence_deepcut'] = result_df['pred_sentence'].map(lambda x: ' '.join(deepcut.tokenize(x)))
result_df['pred_sentence_deepcut_corrected'] = result_df['pred_sentence_corrected'].map(lambda x: ' '.join(deepcut.tokenize(x)))
# result_df.to_csv('artifacts/result_cv70.csv',index=False)

In [3]:
#load our results
result_df = pd.read_csv('artifacts/result_cv70.csv')

In [4]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.13634023935143483

In [44]:
#wer deepcut
wer_metric.compute(predictions=result_df.pred_sentence_deepcut,references=result_df.sentence_deepcut)

0.0815205202960305

In [50]:
#wer deepcut
wer_metric.compute(predictions=result_df.pred_sentence_deepcut_corrected,references=result_df.sentence_deepcut)

0.14167974882260598

In [18]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.028130193905817176

In [51]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence_corrected,references=result_df.sentence)

0.05225761772853186

## Eyeballing Results

In [37]:
pd.set_option('max_colwidth', -1)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == '__main__':


In [47]:
result_df['wer_deepcut'] = result_df.apply(
    lambda row: wer_metric.compute(predictions=[row.pred_sentence_deepcut],
                                   references=[row.sentence_deepcut]),1)

result_df['wer_tok'] = result_df.apply(
    lambda row: wer_metric.compute(predictions=[row.pred_sentence_tok],
                                   references=[row.sentence_tok]),1)

result_df['cer'] = result_df.apply(
    lambda row: cer_metric.compute(predictions=[row.pred_sentence],
                                   references=[row.sentence]),1)

In [60]:
result_df.sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                      'wer_tok','cer']].head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
2195,พระเจ้าวรวงศ์เธอพระองค์เจ้าโสมสวลีพระวรราชาทินัดดามาตุ,พระองค์เจ้าวรวงศ์เธอพระองค์เจ้าโสมสวลีพระวรราชา ที่ นัดดา มาก,4.0,1.166667,0.148148
1978,เบญจางคประดิษฐ์,เบ็น จาง ฆ่า ประดิษฐ์,4.0,4.000000,0.333333
2160,ฟอร์มาลดีไฮด์,เฝ้า มา อดีไฮด,3.0,6.000000,0.461538
2099,แบ่งสันปันส่วน,แบ่งสัน ปั้น ส่วน,3.0,4.000000,0.071429
2400,กระปรี้กระเปร่า,กะ ปรี้ กระเป๋า,3.0,4.000000,0.200000
1545,สหรัฐอาหรับเอมิเรตส์,สหรัฐ อารับ เอมิเรส,3.0,7.000000,0.150000
2325,พุ่มข้าวบิณฑ์,พุ้ง เข้า บิน,3.0,3.000000,0.538462
2265,โรงเรียนวัดบวรมงคล,โรงเรียน วัดบวรณมงคล,2.0,5.000000,0.055556
1572,ปลาทูเจี๋ยน,ประตู เจียน,2.0,1.000000,0.363636
2428,พรีแคลมป์เชีย,พรีแคมป์ เชียร์,2.0,4.000000,0.230769


In [59]:
result_df.sort_values('cer',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                              'wer_tok','cer']].head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
1455,ฟรังก์,ฟรอม,1.000000,2.000000,0.666667
2325,พุ่มข้าวบิณฑ์,พุ้ง เข้า บิน,3.000000,3.000000,0.538462
2219,ดิอิมพอสสิเบิ้ล,ดี อิมพอร์ทซิเบิลส์,2.000000,5.000000,0.533333
1286,รูเปียห์,กรูเปี๊ยะ,1.000000,1.000000,0.500000
2160,ฟอร์มาลดีไฮด์,เฝ้า มา อดีไฮด,3.000000,6.000000,0.461538
2240,ชันสูตร พลิก ศพ,ฉันอะสูท พลิกศบ,1.000000,5.000000,0.461538
2481,ลาโชว์ มอญ,ราโชมอร,1.000000,1.000000,0.444444
2363,ฤทธิยะวรรณาลัย,วิทยวรณาลัย,1.000000,4.000000,0.428571
1835,วัย อลเวงชลาลัย,ไวอนรเวนชลาลัย,1.000000,1.333333,0.428571
1871,เก็บ ก่อน ถ่าย,เก็บกอน ไทน์,1.000000,1.333333,0.416667


In [58]:
result_df[result_df.wer_deepcut<0.9]\
    .sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                 'wer_tok','cer']]\
    .head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
2237,พี่น้อง เซียว ฮื่อ ยี้ ชื่อ ฮวย บ่อ ข่วย,พี่น้องเซียว คื้อยี้ ชื่อ ฮวยบ่อ ควย,0.875000,0.250000,0.121212
1137,สุดยอด ชาร์ลีปรบ มือ ด้วย ความ ชื่นชม,สุด ยอด ชาลี ปรบ มือ ด้วย ความ ชื่มชม,0.833333,0.600000,0.093750
544,ตลก มาก แลดดี้ ตลก มาก,ตลก มาก และ ดี ต ลก มาก,0.800000,0.333333,0.111111
2022,พนา ปำ ทำไม จะ ไพล่เผล,พนาปัมทำไม จะ ไพร่เผล,0.800000,0.333333,0.166667
1956,ศร รัก น้ำผึ้ง รจเรข,สอน รักน้ำผึ้ง รจเรข,0.750000,0.250000,0.176471
2416,สุพรรณบุรี เอฟซีปะทะ สุราษฎร์ธานี โรเวอร์,สุพรรณบุรี เอฟซีปะทสุราธานีโรเวอร์,0.750000,0.571429,0.131579
138,เฮ็ด ใน สิ่ง เซื่อเซื่อ ใน สิ่ง ที่ เฮ็ด,เห็ด ใน สิ่ง สี เสื้อ เสื้อ ใน สิ่ง สี เห็ด,0.750000,0.750000,0.303030
664,ฉัน สงสาร เธอ เหลือเกิน,ฉัน ตอง สาร เธอ หลือเกิน,0.750000,1.250000,0.150000
1523,วิหคที่ หลง ทาง กฤษณาอโศกสิน,วิหก ที่ หลง ทาง ปริษณาอโศกสิน,0.750000,0.666667,0.160000
1547,โสน บานเช้า คัดเค้า บานเย็นศรีฟ้าลดาวัลย์,โสน บานเช้า คัดเค้า บานเย็น สี ฟ้าลดาวัลย์,0.750000,0.142857,0.052632


In [66]:
result_df[result_df.wer_deepcut<0.7]\
    .sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                 'wer_tok','cer']]\
    .head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
1948,การ แผ่ อาณาเขต,การ แผ่อณาเขต,0.666667,4.000000,0.076923
696,ซื้อ หนึ่ง แถมสี่,ซื้อ หนึ่ง แถม สี,0.666667,0.250000,0.066667
1647,ความ สัตย์ จริง,ความ เสรัจจริง,0.666667,4.000000,0.384615
360,ฉัน ทานซีเรียล วีททบิกซ์ เป็น อาหาร เช้า,ฉัน ทาน ซีเรียล วิด บิก เป็น อาหาร เช้า,0.666667,0.300000,0.142857
1497,มอง ตา ไม่กะพริบ,มอง ตา ไม่ กระพริบ,0.666667,2.000000,0.071429
1983,พราหมณ์ ขาย เมีย,คราม ขาย เงีย,0.666667,3.000000,0.357143
1635,ผู้ แทน จำหน่าย,ตัวแทน จำหน่าย,0.666667,1.000000,0.230769
1329,อ้วกว่ะ คือ เมา,อวก ว่ะ คือ เมา,0.666667,0.250000,0.076923
2100,ภวินสัน หลัง วาบ,พวิน สัน หลัง วาบ,0.666667,0.250000,0.071429
2162,กระเป๋า รถ เมล์,กระเป๋า รึกเน,0.666667,3.000000,0.384615


In [61]:
result_df[result_df.wer_deepcut<0.5]\
    .sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                 'wer_tok','cer']]\
    .head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
759,เรา พบ ว่า บุรุษ ไปรษณีย์ ออก จะ ระแวง สุนัขลาบราดอร์ ของ เร่,เรา พบ ว่า บุรุษ ไปรษณีย์ ออก จาก ระแวน สุนัข ราบราดอร์ ของ เร่ต์,0.454545,0.800000,0.117647
737,ดอทรีโชว์ กลการ นับ ด้วย การ พา ค็อกกี้ ไป ด้วย,ดอ์ทรี่ โชว์ กลการ นับ ด้วย การ พา คอก กี้ ไป ด้วย,0.444444,0.250000,0.076923
1181,เค้า กำลัง ทำ ความ เข้าใจ กับ ภาพลักษณ์,เขา กำลัง ทำ ความ เข้าใจ กับ ภาพ ลัก์,0.428571,0.800000,0.121212
882,รัฐบาล สั่ง ห้าม การ ส่ง ออก ปู,รัฐบาล สั่ง ห้าม การ จงออกปู,0.428571,0.750000,0.080000
1483,ยา สี ฟัน คอลเกต วิเศษ นิยม ยา อม หมอชิต เป็น หิดเป็นเหา เป็นเลา เป็น โลน ไม่ ต้อง ไป โกน ใช้ ซีม่า โลชั่น,ยา สี ฟัน คอลเกรจวิเศษนิยม ยา อม หมอชิต เป็น หิต เป็นเหา เป็น เรา เป็น โรน ไม่ ต้อง ไป โกน ใช้ ซีมา โลชั่น,0.428571,0.409091,0.069767
1166,ว้าว คุณ ไป ไว มาก บน ซิปไลน์,ว้าว คุณ ไป ไว้ มาก บน ซิบ ลาย,0.428571,0.375000,0.217391
599,ดู สิ จอห์นฉัน ไม่ สามารถ ซัพพอร์ต ได้,ดู สิ จอ์นฉัน ไม่ สามารถ แสบ พอร์ต ได้,0.428571,0.500000,0.125000
504,กระปุก ออม สิน หาย ไป ไหน นะ,ตระปกออมสิน หาย ไป ไหน นะ,0.428571,0.600000,0.090909
2155,ถ้า ไม่ มี ข้าว กิน ทำไม ไม่ กิน เค้ก ละ,ทำไม มี ข้าว กิน ทำไม ไม่ กิน เค้กรอ,0.400000,0.300000,0.193548
2156,ผง น้ำตาล เกลือแร่ แก้ ท้องเสีย,ผม น้ำตาล เกอแร่ แก้ ท้องเสีย,0.400000,0.800000,0.111111


In [62]:
result_df[result_df.wer_deepcut<0.3]\
    .sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                 'wer_tok','cer']]\
    .head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
1462,แจ็คยอม รับ ว่า เจ้า กิ้งก่า เปิด มิติ ใหม่ แห่ง การ ซาวนา ใน ครั้ง นี้,แจ็คยอม รับ ว่า เจ้า กิ้งกาเปิด มิติ ใหม่ แห่ง การ ชาว น่า ใน ครั้ง นี้,0.285714,0.285714,0.051724
1177,เส้น ผ่า ศูนย์กลาง ไม่ เกิน แปด ยาร์ด,เส้น ผ้า ศูนย์กลาง ไม่ เกิน แปด ยาด,0.285714,0.833333,0.096774
2470,ต้อง ใช้ ความ อดทน ใน การ ประนีประนอม,ต้อง ใช้ ความ อดทน ใน การ ประหนี ประนอม,0.285714,0.800000,0.032258
46,เวลา พลบค่ำ ทำ ให้ คุณ รู้สึก เศร้า,เวลา พบ ค่ำ ทำ ให้ คุณ รู้สึก เศร้า,0.285714,0.333333,0.034483
1205,อีสเทอร์ คุณ กำลัง จะ ไป ที่ ไหน,อีสเธอร์คุณ กำลัง จะ ไป ที่ ไหน,0.285714,0.375000,0.038462
1046,เธอ ไม่ เกรงกลัว ฉัน ใช่ มั้ย ทอม,เธอ ไม่ เกรงกรัว ฉัน ใช่ มั๊ย ทอม,0.285714,0.571429,0.074074
557,แยก อะตอม ไฮโดรเจน ออก จาก โมเลกุล ไซยาไนด์,แยกตะตอม ไฮโดรเจน ออก จาก โมเลกุล ไซยาไนด์,0.285714,0.333333,0.027027
1056,น้ำ ตก ธรรมชาติ ที่ มี น้ำ ใส ไหล เย็น ลดหลั่น กัน เป็น ชั้น ชั้น,น้ำ ตก ธรรมชาติ ที่ มี น้ำ ใส หลา เย็น ลด หล่น กัน เป็น ฉัน ชั้น,0.285714,0.307692,0.096154
1062,จะ เร็ว และ ลึกซึ้ง อะไร ขนาด นั้น,จะ เร็ว ละลึกซึ้ง อะไร ขนาด นั้น,0.285714,0.142857,0.035714
620,บาง ที เรา ควร เข้า ร่วม ชมรมไวน์,บาง ที เรา ควร เข้า ร่วม ชมลม วาย,0.285714,0.500000,0.148148


In [63]:
result_df[result_df.wer_deepcut<0.1]\
    .sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                 'wer_tok','cer']]\
    .head(20)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
2213,มัน เป็น เรื่อง ที่ น่า เบิกบานใจ ที่ ได้ เดิน ออก มา ข้าง นอก ใน สภาพ อากาศ ที่ ปลอดโปร่ง อีก ครั้ง หนึ่ง,มัน เป็น เรื่อง ที่ น่า เบิกบานใจ ที่ ได้ เดิน ออก มา ข้าง นอก ใน สภาพ อากาศ ที่ ปลอด โปร่ อีก ครั้ง หนึ่ง,0.095238,0.187500,0.011628
551,เธอ ใช้ แต่ สี เข้ม เพราะ เธอ ไม่ ชอบ สี อ่อน,เธอ ใช้ กับ สี เข้ม เพราะ เธอ ไม่ ชอบ สี อ่อน,0.090909,0.200000,0.085714
2204,โจทก์ มี สิทธิ ยื่น คำ ร้อง ขอ แก้ไข คำ ฟ้อง ได้,โจทก์ มี สิทธิ ยื่น คำ ร้อง ขอ แก้ไข คำ ฟอง ได้,0.090909,0.250000,0.026316
1283,ชา ที่ ถูก เสิร์ฟ ใน เหยือก สี น้ำตาล มี รสชาติ ดี,ชา ที่ ถูก เสิร์ฟ ใน เหยื่อ สี น้ำตาล มี รสชาติ ดี,0.090909,0.100000,0.050000
527,ตอน นั้น เธอ รู้สึก ได้ ว่า เธอ มี สัมผัส ที่ หก,ตอน นั้น เธอ รู้สึก ได้ ว่า เธอ มี สัมผัส ที่ โหด,0.090909,0.222222,0.052632
1324,เธอ ทำ การ บ้าน และ ฟัง เพลง ไป พร้อม พร้อม กัน,เธอ ทำ การ บ้าน และ ฟัง เพลง ไป พ้ำ พร้อม กัน,0.090909,0.142857,0.081081
725,มัน เป็น เรื่อง ที่ ยาก ที่ จะ ค้นหา ที่ อยู่ จาก ข้อมูล และ คำ อธิบาย อัน คลุม เครือ ที่ พวก เรา มี,มัน เป็น เรื่อง ยาก ที่ จะ ค้นหา ที่ อยู่ จาก ข้อมูล และ คำ อธิบาย อัน คุม เครือ ที่ พวก เรา มี,0.090909,0.187500,0.050633
213,ใน ที่สุด ก็ มี คน คิด รหัส ผ่าน ของ คอมพิวเตอร์ นี้,ใน ที่สุด ก็ มี คน คิด รหัส ผ่าน ก่อ คอมพิวเตอร์ นี้,0.090909,0.111111,0.071429
47,เขา เปิด ห่อ ออก มา และ ใส่ แซน วิช ลง ไป,เขา เปิด ห่อ ออก มา และ ใส่ แชน วิช ลง ไป,0.090909,0.083333,0.032258
807,เป้าหมาย รอง ของ เรา คือ สร้าง สิ่ง ที่ เกิด ขึ้น ใหม่,เป้าหมาย รอง ของ เรา คือ ชร่าง สิ่ง ที่ เกิด ขึ้น ใหม่,0.090909,0.200000,0.045455


In [70]:
result_df[result_df.wer_deepcut==0]\
    .sort_values('wer_deepcut',ascending=False)[['sentence_deepcut','pred_sentence_deepcut','wer_deepcut',
                                                 'wer_tok','cer']]\
    .sample(frac=1).head(25)

,sentence_deepcut,pred_sentence_deepcut,wer_deepcut,wer_tok,cer
916,บาง คน ทำ ผิดพลาด ซ้ำแล้วซ้ำอีก,บาง คน ทำ ผิดพลาด ซ้ำแล้วซ้ำอีก,0.0,0.0,0.0
1215,ให้ พวก เรา นั่ง ลง,ให้ พวก เรา นั่ง ลง,0.0,0.0,0.0
591,ฉัน รู้สึก มั่นใจ ว่า ทักษะ ของ ฉัน นั้น สอดคล้อง กับ ลักษณะ ของ งาน,ฉัน รู้สึก มั่นใจ ว่า ทักษะ ของ ฉัน นั้น สอดคล้อง กับ ลักษณะ ของ งาน,0.0,0.0,0.0
2362,ให้ สามารถ ใช้ ประโยชน์ จาก ความ ก้าวหน้า ทาง เทคโนโลยี,ให้ สามารถ ใช้ ประโยชน์ จาก ความ ก้าวหน้า ทาง เทคโนโลยี,0.0,0.0,0.0
1418,มัน เป็น รูป สี่ เหลี่ยม ผืนผ้า,มัน เป็น รูป สี่ เหลี่ยม ผืนผ้า,0.0,0.0,0.0
2005,ธนบัตร ราคา หนึ่ง ร้อย บาท,ธนบัตร ราคา หนึ่ง ร้อย บาท,0.0,0.0,0.0
1432,ประตู ที่ ใช้ การ ไม่ ได้ บน เรือ ดำ น้ำ,ประตู ที่ ใช้ การ ไม่ ได้ บน เรือ ดำ น้ำ,0.0,0.0,0.0
1097,ไม่ มี คน โง่ อย่าง เช่น คน แก่,ไม่ มี คน โง่ อย่าง เช่น คน แก่,0.0,0.0,0.0
1279,ก็ ทุก การ ช่วยเหลือ เล็ก เล็ก,ก็ ทุก การ ช่วยเหลือ เล็ก เล็ก,0.0,0.0,0.0
2192,หนาว แปร สุภาวดี,หนาว แปร สุภาวดี,0.0,0.0,0.0
